# The Battle of Neighborhoods | Business Proposal | Introduction

## 1-Introduction:

I North Africa There is a big confusion in the migration/tourist communities community for more than 10 years (2000-2010) big focus on the tourist to France but from 2016 there are new calls to migrate to Canada but the community they take serious and even today in covid-19 time they go in illegal migration to France. So we ant to help them see to the evolution of Canada as we are taking from the example of Tronto database and we will compare to Paris.

Toronto and Paris are quite popular tourist and vacation destinations for people all around the world. They are diverse and multicultural and offer a wide variety of experiences that is widely sought after. We try to group the neighbourhoods of Toronto and Paris respectively and draw insights to what they look like now.

## 2-Business Problem:

The aim is to help North african and other tourists to choose their destinations depending on the experiences that the neighbourhoods have to offer and what they would want to have. This also helps people make decisions if they are thinking about migrating to Toronto or Paris or even if they want to relocate neighbourhoods within the city. Our findings will help stakeholders make informed decisions and address any concerns they have including the different kinds of cuisines, provision stores and what the city has to offer.

## 3-Data Description:

We require geographical location data for both Toronto and Paris. Postal codes in each city serve as a starting point. Using Postal codes we can find out the neighborhoods, boroughs, venues and their most popular venue categories.

 **- Toronto:**

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. 

Data Link: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M 

from week 3. Dataset consisting of latitude and longitude, zip codes.

 **- Paris:**

To derive our solution, We leverage JSON data available at https://www.data.gouv.fr/fr/datasets/r/e88c6fda-1d09-42a0-a069-606d3259114e

The JSON file has data about all the neighbourhoods in France, we limit it to Paris.

1. postal_code : Postal codes for France
2. nom_comm : Name of Neighbourhoods in France
3. nom_dept : Name of the boroughs, equivalent to towns in France
4. geo_point_2d : Tuple containing the latitude and longitude of the Neighbourhoods.

**-Foursquare API Data:**

We will need data about different venues in different neighbourhoods of that specific borough. In order to gain that information we will use "Foursquare" locational information. Foursquare is a location data provider with information about all manner of venues and events within an area of interest. Such information includes venue names, locations, menus and even photos. As such, the foursquare location platform will be used as the sole data source since all the stated required information can be obtained through the API.

After finding the list of neighbourhoods, we then connect to the Foursquare API to gather information about venues inside each and every neighbourhood. For each neighbourhood, we have chosen the radius to be 500 meters.

The data retrieved from Foursquare contained information of venues within a specified distance of the longitude and latitude of the postcodes. The information obtained per venue as follows:

1. Neighbourhood : Name of the Neighbourhood
2. Neighbourhood Latitude : Latitude of the Neighbourhood
3. Neighbourhood Longitude : Longitude of the Neighbourhood
4. Venue : Name of the Venue
5. Venue Latitude : Latitude of Venue
6. Venue Longitude : Longitude of Venue
7. Venue Category : Category of Venue

Based on all the information collected for both Tornto and Paris, we have sufficient data to build our model. We cluster the neighbourhoods together based on similar venue categories. We then present our observations and findings. Using this data, our stakeholders can take the necessary decision.

## 4-Methodology:

We will be creating our model with the help of Python so we start off by importing all the required packages.

In [1]:
!pip install folium 
!pip install sklearn

In [2]:
!pip install geocoder

In [3]:
import pandas as pd
import requests
import numpy as np
import geocoder
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
from sklearn.cluster import KMeans

Package to use:

- Pandas : To collect and manipulate data in JSON and HTMl and then data analysis
- requests : Handle http requests
- matplotlib : Detailing the generated maps
- folium : Generating maps of London and Paris
- sklearn : To import Kmeans which is the machine learning model that we are using.

The approach taken here is to explore each of the cities individually, plot the map to show the neighbourhoods being considered and then build our model by clustering all of the similar neighbourhoods together and finally plot the new map with the clustered neighbourhoods. We draw insights and then compare and discuss our findings.

### **1. Toronto:**

To collect data for Toronto, we scrape the List of areas of Toronto wikipedia page from week 3. see following code:


#### Data Collection,Feature Selection and Engineering
In the data collection stage, we begin with collecting the required data for the cities of Toronto and Paris. We need data that has the postal codes, neighbourhoods and boroughs specific to each of the cities.

In [4]:
html = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
wiki_p = requests.get(html)
wiki_raw = pd.read_html(wiki_p.content, header=0)[0]
df_toronto = wiki_raw[wiki_raw.Neighbourhood != 'Not assigned']
df_toronto.reset_index(inplace=True)
df_toronto.groupby(['Postal Code']).first()
print(df_toronto.shape)
df_toronto.head()

(103, 4)


,index,Postal Code,Borough,Neighbourhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Now let's add the geolocalisation using the csv file from week3 :

In [5]:
url = 'https://cocl.us/Geospatial_data'
df_geo = pd.read_csv(url)
df_toronto = df_toronto.join(df_geo.set_index('Postal Code') , on='Postal Code')


In [6]:
df_toronto.reset_index()
df_toronto.drop(['index'], axis='columns', inplace=True)

#### Visualize the Map of Toronto

To help visualize the Map of Toronto and the neighbourhoods in Toronto, we make use of the folium package.

In [7]:
import geocoder
from geopy.geocoders import Nominatim

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geo cordinate of Tronto are {} and {}.'.format(latitude, longitude))

The geo cordinate of Tronto are 43.6534817 and -79.3839347.


In [8]:
# Create a map of Toronto using Latitude and Longitude
map_Tor = folium.Map(location=[latitude, longitude], zoom_start=10)

# adding markers to the map
for lat, long, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html= True)
    folium.CircleMarker([lat, long],radus=5, popup=label,
                      color='green',fill=True, fill_color='green',
                      fill_opacity=0.7,).add_to(map_Tor)
map_Tor

Venues in Toronto

To proceed with the next part, we need to define Foursquare API credentials.

Using Foursquare API, we are able to get the venue and venue categories around each neighbourhood in Toronto.

In [9]:
CLIENT_ID = 'S1ESACNQKERLLG0CGMKJITXLT40LJ2MNPYAM42W4ABUHSUI5'
CLIENT_SECRET = 'FQ4DFOKL5TLUDCYSQYLD5UZJPAFK0UTHEZNSOQ34RBQ0GQUK'
VERSION = '20180604'
print('Your credentials:')
print('CLIENT_ID:', CLIENT_ID)
print('CLIENT_SECRET:', CLIENT_SECRET)

Your credentials:
CLIENT_ID: S1ESACNQKERLLG0CGMKJITXLT40LJ2MNPYAM42W4ABUHSUI5
CLIENT_SECRET: FQ4DFOKL5TLUDCYSQYLD5UZJPAFK0UTHEZNSOQ34RBQ0GQUK


In [10]:
neighborhood_latitude = df_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto.loc[0, 'Longitude'] #neighborhood longitude value

neighborhood_name = df_toronto.loc[0, 'Neighbourhood'] # nei_ name
print('Lat and long values of {} are {} and {}'.format(neighborhood_name, neighborhood_latitude,
                                                      neighborhood_longitude))

Lat and long values of Parkwoods are 43.7532586 and -79.3296565


In [11]:
LIMIT =100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=S1ESACNQKERLLG0CGMKJITXLT40LJ2MNPYAM42W4ABUHSUI5&client_secret=FQ4DFOKL5TLUDCYSQYLD5UZJPAFK0UTHEZNSOQ34RBQ0GQUK&v=20180604&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [12]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5feb3a3c1a514c11a7f7c0ce'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

In [13]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
    if len(categories_list ) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
import json
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues) # flatten json

#fliter columns
filtered_columns = ['venue.name', 'venue.categories','venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Brookbanks Pool,Pool,43.751389,-79.332184
2,Variety Store,Food & Drink Shop,43.751974,-79.333114


Let's get all the venues were returned by Foursquare

In [15]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'], latitudes=df_toronto['Latitude'], longitudes=df_toronto['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

Sampling our data

In [18]:
print(toronto_venues.shape)
toronto_venues.head()

(2122, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Brookbanks Pool,43.751389,-79.332184,Pool
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


Wow, we have scraped together 2122 records for venues. This will definitely make the clustering interesting.

**Grouping by Venue Categories**

We need to now see how many Venue Categories are there for further processing

In [19]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,58,58,58,58,58,58
"Birch Cliff, Cliffside West",5,5,5,5,5,5
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16


In [20]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 267 uniques categories.


There are 267 uniques categories.

**One Hot Encoding**

We need to Encode our venue categories to get a better result for our clustering

In [21]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Truck Stop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
toronto_onehot.shape #examening the niew df size

(2122, 268)

In [23]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Truck Stop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0000,0.0000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,"Alderwood, Long Branch",0.0,0.0000,0.0000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0000,0.0000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
3,Bayview Village,0.0,0.0000,0.0000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
4,"Bedford Park, Lawrence Manor East",0.0,0.0000,0.0000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
5,Berczy Park,0.0,0.0000,0.0000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.0,0.0,0.017241,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
6,"Birch Cliff, Cliffside West",0.0,0.0000,0.0000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
7,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0000,0.0000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
8,"Business reply mail Processing Centre, South C...",0.0,0.0000,0.0000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.062500
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.0000,0.0000,0.062500,0.0625,0.0625,0.125,0.1875,0.125,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


In [24]:
toronto_grouped.shape

(96, 268)

Let's print each neighborhood along with the top 5 most common venues

In [25]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0               Skating Rink  0.25
1                     Lounge  0.25
2             Breakfast Spot  0.25
3  Latin American Restaurant  0.25
4  Middle Eastern Restaurant  0.00


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.25
1        Pharmacy  0.12
2    Skating Rink  0.12
3  Sandwich Place  0.12
4             Pub  0.12


----Bathurst Manor, Wilson Heights, Downsview North----
           venue  freq
0    Coffee Shop  0.09
1           Bank  0.09
2     Restaurant  0.05
3  Shopping Mall  0.05
4           Park  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1  Japanese Restaurant  0.25
2   Chinese Restaurant  0.25
3                 Bank  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.13
1      Sandwich Place  0.09
2         Coffee Shop  0.09
3            Pharmacy  0.04
4    Sushi Rest

                             venue  freq
0                   Baseball Field   1.0
1                Accessories Store   0.0
2              Moroccan Restaurant   0.0
3              Monument / Landmark   0.0
4  Molecular Gastronomy Restaurant   0.0


----Humewood-Cedarvale----
               venue  freq
0              Field  0.25
1         Playground  0.25
2       Hockey Arena  0.25
3              Trail  0.25
4  Accessories Store  0.00


----India Bazaar, The Beaches West----
                  venue  freq
0  Fast Food Restaurant  0.10
1                  Park  0.10
2        Sandwich Place  0.10
3             Pet Store  0.05
4                   Pub  0.05


----Kennedy Park, Ionview, East Birchmount Park----
                        venue  freq
0                  Playground  0.25
1            Department Store  0.25
2                 Bus Station  0.25
3                 Coffee Shop  0.25
4  Modern European Restaurant  0.00


----Kensington Market, Chinatown, Grange Park----
                    

                venue  freq
0         Coffee Shop  0.13
1        Liquor Store  0.07
2  Athletics & Sports  0.07
3                 Pub  0.07
4          Restaurant  0.07


----The Annex, North Midtown, Yorkville----
            venue  freq
0  Sandwich Place  0.14
1            Café  0.14
2     Coffee Shop  0.10
3        Pharmacy  0.05
4     Pizza Place  0.05


----The Beaches----
               venue  freq
0       Neighborhood   0.2
1                Pub   0.2
2  Health Food Store   0.2
3   Asian Restaurant   0.2
4              Trail   0.2


----The Danforth West, Riverdale----
                    venue  freq
0        Greek Restaurant  0.19
1      Italian Restaurant  0.07
2             Coffee Shop  0.07
3  Furniture / Home Store  0.05
4               Bookstore  0.05


----The Kingsway, Montgomery Road, Old Mill North----
                 venue  freq
0                River   0.5
1                 Park   0.5
2    Accessories Store   0.0
3   Miscellaneous Shop   0.0
4  Moroccan Restaurant   0


Let's put that into a pandas dataframe

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Skating Rink,Breakfast Spot,Lounge,Yoga Studio,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Gym,Pharmacy,Skating Rink,Sandwich Place,Pub,Diner,Department Store,Dessert Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Diner,Supermarket,Mobile Phone Shop,Sushi Restaurant,Ice Cream Shop,Shopping Mall,Restaurant,Deli / Bodega
3,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Yoga Studio,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Pizza Place,Restaurant,Café,Sushi Restaurant,Pub,Liquor Store,Thai Restaurant


**Cluster Neighborhoods**

In [28]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 0, 0, 4, 0, 4, 4, 4, 4, 4], dtype=int32)

In [29]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [30]:
toronto_merged = df_toronto

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,Park,Pool,Food & Drink Shop,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Portuguese Restaurant,Hockey Arena,Pizza Place,Coffee Shop,French Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4.0,Coffee Shop,Pub,Bakery,Park,Café,Theater,Breakfast Spot,French Restaurant,Health Food Store,Historic Site
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,4.0,Clothing Store,Accessories Store,Furniture / Home Store,Coffee Shop,Boutique,Miscellaneous Shop,Event Space,Vietnamese Restaurant,Concert Hall,Dim Sum Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0.0,Coffee Shop,Sushi Restaurant,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Portuguese Restaurant,Park


In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

toronto_merged_nonan = toronto_merged.dropna(subset=['Cluster Labels'])

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_nonan['Latitude'], toronto_merged_nonan['Longitude'], toronto_merged_nonan['Neighbourhood'], toronto_merged_nonan['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

**Examine clusters**

Cluster 1

In [32]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 0, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0.0,Portuguese Restaurant,Hockey Arena,Pizza Place,Coffee Shop,French Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
4,Downtown Toronto,0.0,Coffee Shop,Sushi Restaurant,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Portuguese Restaurant,Park
8,East York,0.0,Pizza Place,Gym / Fitness Center,Athletics & Sports,Gastropub,Furniture / Home Store,Intersection,Flea Market,Pet Store,Café,Breakfast Spot
17,Etobicoke,0.0,Pet Store,Coffee Shop,Beer Store,Shopping Plaza,Liquor Store,Café,Pharmacy,Pizza Place,Comfort Food Restaurant,Dessert Shop
22,Scarborough,0.0,Coffee Shop,Pharmacy,Korean BBQ Restaurant,Escape Room,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Department Store,Donut Shop
24,Downtown Toronto,0.0,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Thai Restaurant,Modern European Restaurant,Juice Bar
28,North York,0.0,Bank,Coffee Shop,Diner,Supermarket,Mobile Phone Shop,Sushi Restaurant,Ice Cream Shop,Shopping Mall,Restaurant,Deli / Bodega
29,East York,0.0,Indian Restaurant,Yoga Studio,Supermarket,Grocery Store,Gym,Gas Station,Intersection,Fast Food Restaurant,Discount Store,Park
32,Scarborough,0.0,Grocery Store,Playground,Jewelry Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
38,Scarborough,0.0,Bus Station,Playground,Coffee Shop,Department Store,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center


Cluster 2

In [33]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 1, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,1.0,Cafeteria,Deli / Bodega,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run


Cluster 3

In [34]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 2, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,2.0,Baseball Field,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio,Farm
101,Etobicoke,2.0,Baseball Field,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio,Farm


Cluster 4

In [35]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 3, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,York,3.0,Park,Women's Store,Bar,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
35,East York,3.0,Convenience Store,Park,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
52,North York,3.0,Park,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
64,York,3.0,Park,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
66,North York,3.0,Convenience Store,Park,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
83,Central Toronto,3.0,Gym,Park,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
91,Downtown Toronto,3.0,Park,Playground,Trail,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
98,Etobicoke,3.0,Park,River,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant


Cluster 5

In [36]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 4, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,4.0,Park,Pool,Food & Drink Shop,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
2,Downtown Toronto,4.0,Coffee Shop,Pub,Bakery,Park,Café,Theater,Breakfast Spot,French Restaurant,Health Food Store,Historic Site
3,North York,4.0,Clothing Store,Accessories Store,Furniture / Home Store,Coffee Shop,Boutique,Miscellaneous Shop,Event Space,Vietnamese Restaurant,Concert Hall,Dim Sum Restaurant
6,Scarborough,4.0,Print Shop,Fast Food Restaurant,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
7,North York,4.0,Gym,Restaurant,Japanese Restaurant,Coffee Shop,Beer Store,Art Gallery,Discount Store,Dim Sum Restaurant,Baseball Field,Italian Restaurant
9,Downtown Toronto,4.0,Coffee Shop,Clothing Store,Café,Hotel,Middle Eastern Restaurant,Bubble Tea Shop,Japanese Restaurant,Cosmetics Shop,Diner,Theater
10,North York,4.0,Bakery,Italian Restaurant,Pub,Japanese Restaurant,Yoga Studio,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run
12,Scarborough,4.0,Construction & Landscaping,Bar,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
13,North York,4.0,Gym,Restaurant,Japanese Restaurant,Coffee Shop,Beer Store,Art Gallery,Discount Store,Dim Sum Restaurant,Baseball Field,Italian Restaurant
14,East York,4.0,Park,Beer Store,Dance Studio,Curling Ice,Skating Rink,Yoga Studio,Dog Run,Diner,Discount Store,Distribution Center


**2. Paris:**

To collect data for Paris, we download the JSON file containg all the postal codes of France from https://www.data.gouv.fr/fr/datasets/r/e88c6fda-1d09-42a0-a069-606d3259114e

Using Pandas we load the table after reading the JSON file:

In [37]:
!wget -q -O 'france-data.json' https://www.data.gouv.fr/fr/datasets/r/e88c6fda-1d09-42a0-a069-606d3259114e
print('Data Downloaded!!')
paris_raw = pd.read_json('france-data.json')
paris_raw.head()

Data Downloaded!!


,datasetid,fields,geometry,record_timestamp,recordid
0,correspondances-code-insee-code-postal,"{'code_comm': '645', 'nom_dept': 'ESSONNE', 's...","{'type': 'Point', 'coordinates': [2.2517129721...",2016-09-21T00:29:06.175+02:00,2bf36b38314b6c39dfbcd09225f97fa532b1fc45
1,correspondances-code-insee-code-postal,"{'code_comm': '133', 'nom_dept': 'SEINE-ET-MAR...","{'type': 'Point', 'coordinates': [3.0529405055...",2016-09-21T00:29:06.175+02:00,7ee82e74e059b443df18bb79fc5a19b1f05e5a88
2,correspondances-code-insee-code-postal,"{'code_comm': '378', 'nom_dept': 'ESSONNE', 's...","{'type': 'Point', 'coordinates': [2.1971816504...",2016-09-21T00:29:06.175+02:00,e2cd3186f07286705ed482a10b6aebd9de633c81
3,correspondances-code-insee-code-postal,"{'code_comm': '243', 'nom_dept': 'SEINE-ET-MAR...","{'type': 'Point', 'coordinates': [2.7097808131...",2016-09-21T00:29:06.175+02:00,868bf03527a1d0a9defe5cf4e6fa0a730d725699
4,correspondances-code-insee-code-postal,"{'code_comm': '003', 'nom_dept': 'VAL-DE-MARNE...","{'type': 'Point', 'coordinates': [2.3335102498...",2016-09-21T00:29:06.175+02:00,21e809b1d4480333c8b6fe7addd8f3b06f343e2c


In [38]:
paris_data = pd.DataFrame()
for f in paris_raw.fields:
    dict_new = f
    paris_data = paris_data.append(dict_new, ignore_index=True)
    
paris_data.head()

,code_arr,code_cant,code_comm,code_dept,code_reg,geo_point_2d,geo_shape,id_geofla,insee_com,nom_comm,nom_dept,nom_region,population,postal_code,statut,superficie,z_moyen
0,3,03,645,91,11,"[48.750443119964764, 2.251712972144151]","{'type': 'Polygon', 'coordinates': [[[2.238024...",16275,91645,VERRIERES-LE-BUISSON,ESSONNE,ILE-DE-FRANCE,15.5,91370,Commune simple,999.0,121.0
1,3,20,133,77,11,"[48.41256065214989, 3.052940505560729]","{'type': 'Polygon', 'coordinates': [[[3.076046...",31428,77133,COURCELLES-EN-BASSEE,SEINE-ET-MARNE,ILE-DE-FRANCE,0.2,77126,Commune simple,1082.0,88.0
2,1,09,378,91,11,"[48.52726809075556, 2.19718165044305]","{'type': 'Polygon', 'coordinates': [[[2.203466...",30975,91378,MAUCHAMPS,ESSONNE,ILE-DE-FRANCE,0.3,91730,Commune simple,313.0,150.0
3,5,14,243,77,11,"[48.87307018579678, 2.7097808131278462]","{'type': 'Polygon', 'coordinates': [[[2.727542...",17000,77243,LAGNY-SUR-MARNE,SEINE-ET-MARNE,ILE-DE-FRANCE,20.2,77400,Chef-lieu canton,579.0,71.0
4,3,34,003,94,11,"[48.80588035965699, 2.333510249842654]","{'type': 'Polygon', 'coordinates': [[[2.343851...",32123,94003,ARCUEIL,VAL-DE-MARNE,ILE-DE-FRANCE,19.5,94110,Chef-lieu canton,232.0,70.0


In [39]:
df_paris = paris_data[['postal_code', 'nom_comm', 'nom_dept', 'geo_point_2d']]

In [40]:
df_paris = df_paris[df_paris['nom_dept'].str.contains('PARIS')].reset_index(drop=True)
df_paris.head()

,postal_code,nom_comm,nom_dept,geo_point_2d
0,75009,PARIS-9E-ARRONDISSEMENT,PARIS,"[48.87689616237872, 2.337460241388529]"
1,75002,PARIS-2E-ARRONDISSEMENT,PARIS,"[48.86790337886785, 2.344107166658533]"
2,75011,PARIS-11E-ARRONDISSEMENT,PARIS,"[48.85941549762748, 2.378741060237548]"
3,75015,PARIS-15E-ARRONDISSEMENT,PARIS,"[48.84015541860987, 2.293559372435076]"
4,75019,PARIS-19E-ARRONDISSEMENT,PARIS,"[48.88686862295828, 2.384694327870042]"


In [41]:
df_paris.shape

(20, 4)

In [42]:
df_paris.dtypes

postal_code     object
nom_comm        object
nom_dept        object
geo_point_2d    object
dtype: object

For the geolocation we just need to extract the latitude and longitude for the column:

Checking one of the geo coordinates.

In [43]:
df_paris['geo_point_2d'][0]

[48.87689616237872, 2.337460241388529]

In [44]:
temp1 = df_paris['geo_point_2d']
temp1

0      [48.87689616237872, 2.337460241388529]
1      [48.86790337886785, 2.344107166658533]
2      [48.85941549762748, 2.378741060237548]
3      [48.84015541860987, 2.293559372435076]
4      [48.88686862295828, 2.384694327870042]
5      [48.86318677744551, 2.400819826729021]
6      [48.86305413181178, 2.359361058970589]
7      [48.84896809191946, 2.332670898588416]
8     [48.892735074561706, 2.348711933867703]
9     [48.854228281954754, 2.357361938142205]
10      [48.8626304851685, 2.336293446550539]
11     [48.88733716648682, 2.307485559493426]
12     [48.87252726662346, 2.312582560420059]
13     [48.82871768452136, 2.362468228516128]
14     [48.83515623066034, 2.419807034965275]
15    [48.844508659617546, 2.349859385560182]
16     [48.87602855694339, 2.361112904561707]
17     [48.86039876035177, 2.262099559395783]
18     [48.85608259819694, 2.312438687733857]
19     [48.82899321160942, 2.327100883257538]
Name: geo_point_2d, dtype: object

In [45]:
paris_latlng = df_paris['geo_point_2d'].astype('str')

Splitting the geo_point_2d columns into latitude and longitude.

**Latitude**


In [46]:
paris_lat = paris_latlng.apply(lambda x: x.split(',')[0])
paris_lat = paris_lat.apply(lambda x: x.lstrip('['))
paris_lat

0      48.87689616237872
1      48.86790337886785
2      48.85941549762748
3      48.84015541860987
4      48.88686862295828
5      48.86318677744551
6      48.86305413181178
7      48.84896809191946
8     48.892735074561706
9     48.854228281954754
10      48.8626304851685
11     48.88733716648682
12     48.87252726662346
13     48.82871768452136
14     48.83515623066034
15    48.844508659617546
16     48.87602855694339
17     48.86039876035177
18     48.85608259819694
19     48.82899321160942
Name: geo_point_2d, dtype: object

In [47]:
paris_lng = paris_latlng.apply(lambda x: x.split(',')[1])
paris_lng = paris_lng.apply(lambda x: x.rstrip(']'))
paris_lng

0      2.337460241388529
1      2.344107166658533
2      2.378741060237548
3      2.293559372435076
4      2.384694327870042
5      2.400819826729021
6      2.359361058970589
7      2.332670898588416
8      2.348711933867703
9      2.357361938142205
10     2.336293446550539
11     2.307485559493426
12     2.312582560420059
13     2.362468228516128
14     2.419807034965275
15     2.349859385560182
16     2.361112904561707
17     2.262099559395783
18     2.312438687733857
19     2.327100883257538
Name: geo_point_2d, dtype: object

In [48]:
paris_geo_lat = pd.DataFrame(paris_lat.astype(float))
paris_geo_lat.columns=['Latitude']
paris_geo_lng = pd.DataFrame(paris_lng.astype(float))
paris_geo_lng.columns=['Longitude']

Preparing our combined data by dropping the geo_point_2d column from our previously stored df_paris and concatenating with the latitude and longitude extracted from it

In [49]:
df_paris = pd.concat([df_paris.drop('geo_point_2d', axis=1), paris_geo_lat, paris_geo_lng], axis=1)
df_paris

,postal_code,nom_comm,nom_dept,Latitude,Longitude
0,75009,PARIS-9E-ARRONDISSEMENT,PARIS,48.876896,2.337460
1,75002,PARIS-2E-ARRONDISSEMENT,PARIS,48.867903,2.344107
2,75011,PARIS-11E-ARRONDISSEMENT,PARIS,48.859415,2.378741
3,75015,PARIS-15E-ARRONDISSEMENT,PARIS,48.840155,2.293559
4,75019,PARIS-19E-ARRONDISSEMENT,PARIS,48.886869,2.384694
5,75020,PARIS-20E-ARRONDISSEMENT,PARIS,48.863187,2.400820
6,75003,PARIS-3E-ARRONDISSEMENT,PARIS,48.863054,2.359361
7,75006,PARIS-6E-ARRONDISSEMENT,PARIS,48.848968,2.332671
8,75018,PARIS-18E-ARRONDISSEMENT,PARIS,48.892735,2.348712
9,75004,PARIS-4E-ARRONDISSEMENT,PARIS,48.854228,2.357362


### Visualizing the Neighbourhoods of Toronto and Paris



In [50]:
address = 'Paris, France'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
paris_lat_coords = location.latitude
paris_lng_coords = location.longitude
print('The geo cordinate of Paris are {} and {}.'.format(paris_lat_coords, paris_lng_coords))

#paris_lng_coords = 48.85341000000005
#paris_lat_coords = 2.3488000000000397
#print("The geolocation of Paris: ", paris_lat_coords, paris_lng_coords)

The geo cordinate of Paris are 48.8566969 and 2.3514616.


In [51]:
# Creating the map of Paris
map_Paris= folium.Map(location=[paris_lat_coords, paris_lng_coords], zoom_start=12)
map_Paris

# adding markers to map
for latitude, longitude, borough, town in zip(df_paris['Latitude'], df_paris['Longitude'], df_paris['nom_comm'], df_paris['nom_dept']):
    label = '{}, {}'.format(town, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='Blue',
        fill=True,
        fill_opacity=0.8
        ).add_to(map_Paris)  
    
map_Paris

In [52]:

venues_in_Paris = getNearbyVenues(df_paris['nom_comm'], df_paris['Latitude'], df_paris['Longitude'])

PARIS-9E-ARRONDISSEMENT
PARIS-2E-ARRONDISSEMENT
PARIS-11E-ARRONDISSEMENT
PARIS-15E-ARRONDISSEMENT
PARIS-19E-ARRONDISSEMENT
PARIS-20E-ARRONDISSEMENT
PARIS-3E-ARRONDISSEMENT
PARIS-6E-ARRONDISSEMENT
PARIS-18E-ARRONDISSEMENT
PARIS-4E-ARRONDISSEMENT
PARIS-1ER-ARRONDISSEMENT
PARIS-17E-ARRONDISSEMENT
PARIS-8E-ARRONDISSEMENT
PARIS-13E-ARRONDISSEMENT
PARIS-12E-ARRONDISSEMENT
PARIS-5E-ARRONDISSEMENT
PARIS-10E-ARRONDISSEMENT
PARIS-16E-ARRONDISSEMENT
PARIS-7E-ARRONDISSEMENT
PARIS-14E-ARRONDISSEMENT


Sampling our Data



In [53]:
venues_in_Paris.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,So Nat,48.876277,2.338614,Vegetarian / Vegan Restaurant
1,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,Place Saint-Georges,48.878408,2.337575,Plaza
2,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,Le Bouclier de Bacchus,48.876834,2.337843,Wine Bar
3,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,Farine & O,48.877209,2.339464,Bakery
4,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,La Compagnie du Café,48.877916,2.337997,Café


In [54]:
venues_in_Paris.shape

(1276, 7)

We have managed to collect 1276 venue records for the neighbourhoods in Paris



**Grouping by Venue Categories**

We need to now see how many Venue Categories are there for further processing

In [55]:
venues_in_Paris.groupby('Venue Category').max()


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Afghan Restaurant,PARIS-11E-ARRONDISSEMENT,48.859415,2.378741,Afghanistan,48.862327,2.379999
African Restaurant,PARIS-9E-ARRONDISSEMENT,48.876896,2.361113,Wally Le Saharien,48.879211,2.357852
American Restaurant,PARIS-6E-ARRONDISSEMENT,48.892735,2.384694,Harper's,48.894373,2.379129
Antique Shop,PARIS-9E-ARRONDISSEMENT,48.876896,2.337460,Hôtel des Ventes Drouot,48.873061,2.340101
Argentinian Restaurant,PARIS-3E-ARRONDISSEMENT,48.863054,2.359361,Anahi,48.867002,2.358244
Art Gallery,PARIS-8E-ARRONDISSEMENT,48.872527,2.359361,Galerie Thaddaeus Ropac,48.870204,2.363539
Art Museum,PARIS-8E-ARRONDISSEMENT,48.887337,2.378741,"Pavillon des Sessions – Arts d'Afrique, d'Asie...",48.883155,2.380911
Arts & Crafts Store,PARIS-4E-ARRONDISSEMENT,48.854228,2.357362,Mombini,48.855054,2.355579
Asian Restaurant,PARIS-7E-ARRONDISSEMENT,48.886869,2.384694,Yummy Asian Food,48.883029,2.385464


**One Hot Encoding**

We need to Encode our venue categories to get a better result for our clustering

In [56]:
Paris_venue_cat = pd.get_dummies(venues_in_Paris[['Venue Category']], prefix="", prefix_sep="")
Paris_venue_cat

,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Adding Neighbourhoods to our Data



In [57]:
Paris_venue_cat['Neighbourhood'] = venues_in_Paris['Neighbourhood'] 

# moving neighborhood column to the first column
fixed_columns = [Paris_venue_cat.columns[-1]] + list(Paris_venue_cat.columns[:-1])
Paris_venue_cat = Paris_venue_cat[fixed_columns]

Paris_venue_cat.head()

,Neighbourhood,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
0,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Venue categories mean value**

We will group the Neighbourhoods and calculate the mean venue categories value in each Neighbourhood

In [58]:
Paris_grouped = Paris_venue_cat.groupby('Neighbourhood').mean().reset_index()
Paris_grouped.head()

,Neighbourhood,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
0,PARIS-10E-ARRONDISSEMENT,0.000000,0.02,0.0,0.0,0.0,0.0,0.000000,0.0,0.030000,...,0.01,0.0,0.000000,0.000000,0.0,0.020000,0.02,0.0,0.0,0.0
1,PARIS-11E-ARRONDISSEMENT,0.022727,0.00,0.0,0.0,0.0,0.0,0.022727,0.0,0.045455,...,0.00,0.0,0.022727,0.022727,0.0,0.022727,0.00,0.0,0.0,0.0
2,PARIS-12E-ARRONDISSEMENT,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.2,0.2
3,PARIS-13E-ARRONDISSEMENT,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.213115,...,0.00,0.0,0.000000,0.229508,0.0,0.000000,0.00,0.0,0.0,0.0
4,PARIS-14E-ARRONDISSEMENT,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0


**Top venue categories**

Reusing our previously defined function to get the top venue categories in the neighbourhoods of Paris.

In [59]:
# create a new dataframe for Paris
neighborhoods_venues_sorted_paris = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_paris['Neighbourhood'] = Paris_grouped['Neighbourhood']

for ind in np.arange(Paris_grouped.shape[0]):
    neighborhoods_venues_sorted_paris.iloc[ind, 1:] = return_most_common_venues(Paris_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_paris.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,PARIS-10E-ARRONDISSEMENT,French Restaurant,Bistro,Hotel,Coffee Shop,Café,Italian Restaurant,Asian Restaurant,Indian Restaurant,Japanese Restaurant,Pizza Place
1,PARIS-11E-ARRONDISSEMENT,Restaurant,Italian Restaurant,Café,French Restaurant,Bakery,Asian Restaurant,Pastry Shop,Bar,Mexican Restaurant,Art Museum
2,PARIS-12E-ARRONDISSEMENT,Zoo Exhibit,Supermarket,Monument / Landmark,Bistro,Zoo,Argentinian Restaurant,Exhibit,Frozen Yogurt Shop,French Restaurant,Fountain
3,PARIS-13E-ARRONDISSEMENT,Vietnamese Restaurant,Asian Restaurant,Thai Restaurant,Chinese Restaurant,French Restaurant,Juice Bar,Italian Restaurant,Cambodian Restaurant,Furniture / Home Store,Metro Station
4,PARIS-14E-ARRONDISSEMENT,French Restaurant,Bakery,Japanese Restaurant,Hotel,Bistro,Italian Restaurant,Pizza Place,Fast Food Restaurant,Food & Drink Shop,Brasserie


**K Means**

Let's cluster the city of Paris to roughly 5 to make it easier to analyze.

We use the K Means clustering technique to do so.

In [60]:
# set number of clusters
k_num_clusters = 5

Paris_grouped_clustering = Paris_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans_Paris = KMeans(n_clusters=k_num_clusters, random_state=0).fit(Paris_grouped_clustering)
kmeans_Paris

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

In [61]:
kmeans_Paris.labels_

array([0, 0, 3, 2, 4, 0, 1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 0],
      dtype=int32)

In [62]:
neighborhoods_venues_sorted_paris.insert(0, 'Cluster Labels', kmeans_Paris.labels_ +1)

Join paris_combined_data with our neighbourhood venues sorted to add latitude & longitude for each of the neighborhood to prepare it for plotting

In [63]:

paris_data = df_paris

paris_data = paris_data.join(neighborhoods_venues_sorted_paris.set_index('Neighbourhood'), on='nom_comm')

paris_data.head()

,postal_code,nom_comm,nom_dept,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,75009,PARIS-9E-ARRONDISSEMENT,PARIS,48.876896,2.337460,1,French Restaurant,Hotel,Bakery,Japanese Restaurant,Bistro,Lounge,Wine Bar,Cocktail Bar,Bar,Coffee Shop
1,75002,PARIS-2E-ARRONDISSEMENT,PARIS,48.867903,2.344107,1,French Restaurant,Cocktail Bar,Coffee Shop,Bakery,Wine Bar,Hotel,Indie Movie Theater,Mexican Restaurant,Creperie,Tea Room
2,75011,PARIS-11E-ARRONDISSEMENT,PARIS,48.859415,2.378741,1,Restaurant,Italian Restaurant,Café,French Restaurant,Bakery,Asian Restaurant,Pastry Shop,Bar,Mexican Restaurant,Art Museum
3,75015,PARIS-15E-ARRONDISSEMENT,PARIS,48.840155,2.293559,1,Italian Restaurant,Hotel,French Restaurant,Thai Restaurant,Indian Restaurant,Japanese Restaurant,Lebanese Restaurant,Park,Coffee Shop,Restaurant
4,75019,PARIS-19E-ARRONDISSEMENT,PARIS,48.886869,2.384694,1,French Restaurant,Bar,Supermarket,Hotel,Beer Bar,Seafood Restaurant,Café,Bistro,Canal,Spa


In [64]:
paris_data_nonan = paris_data.dropna(subset=['Cluster Labels'])

In [65]:
map_clusters_paris = folium.Map(location=[paris_lat_coords, paris_lng_coords], zoom_start=12)

# set color scheme for the clusters
x = np.arange(k_num_clusters)
ys = [i + x + (i*x)**2 for i in range(k_num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(paris_data_nonan['Latitude'], paris_data_nonan['Longitude'], paris_data_nonan['nom_comm'], paris_data_nonan['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) +1) + ' ' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.8
        ).add_to(map_clusters_paris)
        
map_clusters_paris

Examining our Clusters

Cluster 1

In [66]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 1, paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]

,nom_comm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,PARIS-9E-ARRONDISSEMENT,1,French Restaurant,Hotel,Bakery,Japanese Restaurant,Bistro,Lounge,Wine Bar,Cocktail Bar,Bar,Coffee Shop
1,PARIS-2E-ARRONDISSEMENT,1,French Restaurant,Cocktail Bar,Coffee Shop,Bakery,Wine Bar,Hotel,Indie Movie Theater,Mexican Restaurant,Creperie,Tea Room
2,PARIS-11E-ARRONDISSEMENT,1,Restaurant,Italian Restaurant,Café,French Restaurant,Bakery,Asian Restaurant,Pastry Shop,Bar,Mexican Restaurant,Art Museum
3,PARIS-15E-ARRONDISSEMENT,1,Italian Restaurant,Hotel,French Restaurant,Thai Restaurant,Indian Restaurant,Japanese Restaurant,Lebanese Restaurant,Park,Coffee Shop,Restaurant
4,PARIS-19E-ARRONDISSEMENT,1,French Restaurant,Bar,Supermarket,Hotel,Beer Bar,Seafood Restaurant,Café,Bistro,Canal,Spa
5,PARIS-20E-ARRONDISSEMENT,1,Plaza,French Restaurant,Bakery,Hotel,Bistro,Japanese Restaurant,Italian Restaurant,Café,Supermarket,Bar
6,PARIS-3E-ARRONDISSEMENT,1,French Restaurant,Coffee Shop,Gourmet Shop,Japanese Restaurant,Italian Restaurant,Bakery,Wine Bar,Art Gallery,Sandwich Place,Cocktail Bar
7,PARIS-6E-ARRONDISSEMENT,1,Chocolate Shop,Bakery,French Restaurant,Pastry Shop,Fountain,Italian Restaurant,Outdoor Sculpture,Seafood Restaurant,Sandwich Place,Café
8,PARIS-18E-ARRONDISSEMENT,1,French Restaurant,Bar,Italian Restaurant,Café,Restaurant,Plaza,Pizza Place,Bistro,Hotel,Convenience Store
9,PARIS-4E-ARRONDISSEMENT,1,French Restaurant,Ice Cream Shop,Park,Clothing Store,Hotel,Wine Bar,Plaza,Pedestrian Plaza,Pastry Shop,Furniture / Home Store


Cluster 2

In [67]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 2, paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]

,nom_comm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,PARIS-16E-ARRONDISSEMENT,2,Plaza,Lake,Pool,Bus Stop,Park,French Restaurant,Boat or Ferry,Art Museum,Bus Station,Farmers Market


Cluster 3

In [68]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 3, paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]

,nom_comm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,PARIS-13E-ARRONDISSEMENT,3,Vietnamese Restaurant,Asian Restaurant,Thai Restaurant,Chinese Restaurant,French Restaurant,Juice Bar,Italian Restaurant,Cambodian Restaurant,Furniture / Home Store,Metro Station


Cluster 4

In [69]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 4, paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]

,nom_comm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,PARIS-12E-ARRONDISSEMENT,4,Zoo Exhibit,Supermarket,Monument / Landmark,Bistro,Zoo,Argentinian Restaurant,Exhibit,Frozen Yogurt Shop,French Restaurant,Fountain


Cluster 5

In [70]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 5, paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]

,nom_comm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,PARIS-17E-ARRONDISSEMENT,5,Hotel,French Restaurant,Italian Restaurant,Café,Bakery,Japanese Restaurant,Plaza,Bistro,Restaurant,Breakfast Spot
12,PARIS-8E-ARRONDISSEMENT,5,French Restaurant,Hotel,Spa,Bakery,Italian Restaurant,Toy / Game Store,Modern European Restaurant,Furniture / Home Store,Sporting Goods Shop,Mediterranean Restaurant
18,PARIS-7E-ARRONDISSEMENT,5,Hotel,French Restaurant,Italian Restaurant,Café,Plaza,History Museum,Cocktail Bar,Bistro,Cheese Shop,Coffee Shop
19,PARIS-14E-ARRONDISSEMENT,5,French Restaurant,Bakery,Japanese Restaurant,Hotel,Bistro,Italian Restaurant,Pizza Place,Fast Food Restaurant,Food & Drink Shop,Brasserie


## Results and Discussion


Toronto's neighborhoods are very multicultural. There are many types of cousins  such as Tunisian, Arabic, Italian, Turkish and Chinese. Toronto seems to take a step further in this direction by offering plenty of restaurants, juice bars, cafes, a Fish and Chips store and breakfast bars. It also has many shopping opportunities with signage. The main modes of transport seem to be Buses and trains. For leisure, there are many parks in the districts. 

Overall, the city of Toronto offers a multicultural, diverse, and certainly fun experience. 

Paris is relatively small geographically. However, there is a wide variety of cuisines and dishes, including French, Tunisian, Algerian, Moroccan, Cambodian, Asian, Chinese and so on. There are many restaurants, including many restaurants. There are many bistros in Paris. Various means of public transport in Paris, including buses, bicycles, boats or ferries. There are many squares, roads, parks, historical sites, clothing stores, art galleries and museums for leisure and sightseeing. Paris seems to be a relaxing holiday destination with a mix of lakes, historical sites and a wide variety of cousins. 

## Conclusion


The aim of this project was to explore the cities of Toronto and Paris and see how attractive it is for tourists and potential travelers. We surveyed the two cities according to their postal codes and then extrapolated common venues locations that were present in each neighborhood to finally sort by similar neighborhoods. 

We have seen that each district in both cities has a diverse experience to offer that is unique in its own way. Cultural diversity is quite noticeable which also gives a sense of a sense of belonging. 

Paris and Toronto seem to offer leisure and beautiful scenery. 

Overall, it is up to stakeholders to decide which experience they prefer and which would be best for them. 